In [ ]:
from bs4 import BeautifulSoup
from time import sleep
from requests.adapters import HTTPAdapter
from random import randint
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# Scraping the data form planecrashinfo.com
date = []
time = []
airline = []
flight = []
location = []
type = []
reg = []
serial = []
operator = []
aboard = []
fatalities = []
ground = []
summary = []
route = []

for year in range (1920, 2024):
    url = 'http://www.planecrashinfo.com/'+str(year)+'/'+str(year)+'.htm'
    print(url)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    tr = soup.find_all('tr')
    number_of_rows = len(tr)

    for i in range(1, number_of_rows):
        url = 'http://www.planecrashinfo.com/'+str(year)+'/'+str(year)+'-'+str(i)+'.htm'
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')

        data = soup.find_all('td', attrs={'align': "left"})

        date.append(data[2].text)
        time.append(data[3].text)
        location.append(data[4].text)
        operator.append(data[5].text)
        flight.append(data[6].text)
        route.append(data[7].text)
        type.append(data[8].text)
        reg.append(data[9].text)
        serial.append(data[10].text)
        aboard.append(data[11].text)
        fatalities.append(data[12].text)
        ground.append(data[13].text)
        summary.append(data[14].text)
    sleep(10)


In [ ]:
planecrashinfo = pd.DataFrame({
    'Date' : date,
    'Time' : time,
    'Location' : location,
    'Operator' : operator,
    'Flight' : flight,
    'Route' : route,
    'Type' : type,
    'Reg' : reg,
    'Serial' : serial,
    'Aboard' : aboard,
    'Fatalities' : fatalities,
    'Ground' : ground,
    'Summary' : summary

})

In [ ]:
display(planecrashinfo)

,Date,Time,Location,Operator,Flight,Route,Type,Reg,Serial,Fatalities,Ground,Summary
0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1 (passengers:0 crew:0),0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5 (passengers:0 crew:5),0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1 (passengers:0 crew:1),0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,14 (passengers:? crew:?),0,The airship flew into a thunderstorm and encou...
...,...,...,...,...,...,...,...,...,...,...,...,...
5022,"July 16, 2022",2247,"Eleftheroupolis, Greece",Meridian,MEM3032,Nis- Amman,Antonov An-12,UR-CIC,01347701,8 (passengers:0 crew:8),0,The cargo plane carrying eight crew members an...
5023,"November 06, 2022",0853,"Bukoba, Tanzania",Precision Air,PW494,Dar es-Salaam -Bukoba,ATR 42-500,5H-PWF,819,19 (passengers:17 crew:2),0,"While on final approach to Bukoba Airport, the..."
5024,"November 18, 2022",1511,"Lima, Peru",LATAM,LA2213,Lima - Juliaca,Airbus 320-271N,CC-BHB,7864,0 (passengers:0 crew:0),2,The Airbus A320 collided with a fire truck whi...
5025,"November 21, 2022",1015,"Medellín, Colombia",AeroPaca SAS,?,Medellín - Pizarro,Piper PA-31-350 Navajo Chieftain,HK-5121,31-7652004,8 (passengers:6 crew:2),0,The plane was chartered to carry a team of six...


In [ ]:
planecrashinfo.to_excel('planecrashinfo.xlsx', index='false')

In [ ]:
df2 = pd.DataFrame(planecrashinfo['Location'])
display(df2)

In [ ]:
#use the location column to get the coordinate for each of the locations in the data
locator = Nominatim(user_agent="myGeocoder")

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df2['location'] = df2['Location'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df2['point'] = df2['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df2[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df2['point'].tolist(), index=df2.index)
df2.head()

RateLimiter caught an error, retrying (0/2 tries). Called with (*('La Rache, Morocco',), **{}).
Traceback (most recent call last):
  File "/Users/prempatel/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/prempatel/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/prempatel/opt/anaconda3/lib/python3.9/http/client.py", line 1371, in getresponse
    response.begin()
  File "/Users/prempatel/opt/anaconda3/lib/python3.9/http/client.py", line 319, in begin
    version, status, reason = self._read_status()
  File "/Users/prempatel/opt/anaconda3/lib/python3.9/http/client.py", line 280, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/Users/prempatel/opt/anaconda3/lib/python3.9/socket.py", line 704, in readinto
    return sel

,Location,location,point,latitude,longitude,altitude
0,"Fort Myer, Virginia","(North Fort Myer Drive, Rosslyn, Arlington, Ar...","(38.8939638, -77.0720349, 0.0)",38.893964,-77.072035,0.0
1,"Juvisy-sur-Orge, France","(Juvisy-sur-Orge, Palaiseau, Essonne, Île-de-F...","(48.6904063, 2.373809, 0.0)",48.690406,2.373809,0.0
2,"Atlantic City, New Jersey","(Atlantic City, Atlantic County, New Jersey, 0...","(39.3642852, -74.4229351, 0.0)",39.364285,-74.422935,0.0
3,"Victoria, British Columbia, Canada","(Victoria, Capital Regional District, British ...","(48.4283182, -123.3649533, 0.0)",48.428318,-123.364953,0.0
4,the North Sea,"(The North Sea, 40, Town Street, Stanningley, ...","(53.807922899999994, -1.6605354963206316, 0.0)",53.807923,-1.660535,0.0


In [ ]:
display(df2)
df2.to_excel('coordinate.xlsx', index=False)

,Location,location,point,latitude,longitude,altitude
0,"Fort Myer, Virginia","(North Fort Myer Drive, Rosslyn, Arlington, Ar...","(38.8939638, -77.0720349, 0.0)",38.893964,-77.072035,0.0
1,"Juvisy-sur-Orge, France","(Juvisy-sur-Orge, Palaiseau, Essonne, Île-de-F...","(48.6904063, 2.373809, 0.0)",48.690406,2.373809,0.0
2,"Atlantic City, New Jersey","(Atlantic City, Atlantic County, New Jersey, 0...","(39.3642852, -74.4229351, 0.0)",39.364285,-74.422935,0.0
3,"Victoria, British Columbia, Canada","(Victoria, Capital Regional District, British ...","(48.4283182, -123.3649533, 0.0)",48.428318,-123.364953,0.0
4,the North Sea,"(The North Sea, 40, Town Street, Stanningley, ...","(53.807922899999994, -1.6605354963206316, 0.0)",53.807923,-1.660535,0.0
...,...,...,...,...,...,...
5022,"Eleftheroupolis, Greece",None,None,NaN,NaN,NaN
5023,"Bukoba, Tanzania","(Bukoba, Kagera, Lake Zone, Tanzania, (-1.3520...","(-1.35209535, 31.686078306387767, 0.0)",-1.352095,31.686078,0.0
5024,"Lima, Peru","(Lima, Lima Metropolitana, Lima, Perú, (-12.06...","(-12.0621065, -77.0365256, 0.0)",-12.062107,-77.036526,0.0
5025,"Medellín, Colombia","(Medellín, Valle de Aburrá, Antioquia, 0500, C...","(6.2443382, -75.573553, 0.0)",6.244338,-75.573553,0.0
